In [24]:
from datasets import load_dataset

ds = load_dataset("Trelis/tiny-shakespeare")

In [27]:
ds_ = ds['train']['Text']

In [29]:
chars = sorted(set(''.join(ds_)))
len(chars)

65

In [37]:
encoder_dict = {k:v for k,v in zip(chars, range(len(chars)))}
decoder_dict = {v:k for k,v in zip(chars, range(len(chars)))}

encode = lambda x: [encoder_dict[letter] for letter in x]
decode = lambda x: ''.join([decoder_dict[letter] for letter in x])
encode('hello'), decode([46, 43, 50, 50, 53])

([46, 43, 50, 50, 53], 'hello')

In [44]:
ds_all = '\n'.join(ds_)
ds_encoded = encode(ds_all)

In [49]:
n = int(.9*len(ds_encoded))
train_data = ds_encoded[:n]
val_data = ds_encoded[n:]